In [5]:
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)

# Categorical Variables

In [6]:
usaid = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0, low_memory= False)
usaid = usaid[["Average Days Late", "Country", "Product Category", "Latest Actual Delivery Date",
               "Latest Actual Delivery Date Fiscal Quarter Year", "Shipped Quantity",
               "Transportation Mode", "Order Type", "Order Entry Date"]]
usaid.dropna(inplace= True)
usaid.isna().sum()

Average Days Late                                  0
Country                                            0
Product Category                                   0
Latest Actual Delivery Date                        0
Latest Actual Delivery Date Fiscal Quarter Year    0
Shipped Quantity                                   0
Transportation Mode                                0
Order Type                                         0
Order Entry Date                                   0
dtype: int64

In [7]:
# Average Days Late vs Product Category:
pc_vs_adl = usaid[["Average Days Late", "Product Category", "Latest Actual Delivery Date Fiscal Quarter Year"]]
pc_vs_adl[["Latest Actual Delivery Date Fiscal Quarter Year", "Quarter"]] = \
  pc_vs_adl["Latest Actual Delivery Date Fiscal Quarter Year"].str.split("-", expand= True)
pc_vs_adl.drop("Latest Actual Delivery Date Fiscal Quarter Year", axis= 1, inplace= True)

# Pick the five most frequent Product Category
most_frequent_categories = pc_vs_adl["Product Category"].value_counts()[:5].index.to_list()
pc_vs_adl = pc_vs_adl[pc_vs_adl["Product Category"].isin(most_frequent_categories)]

# Subset by quarter:
def transform(df: pd.DataFrame, label: str, by_quarter: bool = False, quarter: str = "", *args, **kwargs):
  if by_quarter == True:
    df = df[df["Quarter"] == quarter]
    df.drop("Quarter", axis= 1, inplace= True)
    df = df.pivot(columns= label).stack(level= 0).fillna(0).reset_index(drop= True)
  else:
    df.drop("Quarter", axis= 1, inplace= True)
    df = df.pivot(columns= label).stack(level= 0).fillna(0).reset_index(drop= True)
  return df

pc_vs_adl_Q1 = transform(pc_vs_adl, by_quarter= True, quarter= "Q1", label= "Product Category")
pc_vs_adl_Q2 = transform(pc_vs_adl, by_quarter= True, quarter= "Q2", label= "Product Category")
pc_vs_adl_Q3 = transform(pc_vs_adl, by_quarter= True, quarter= "Q3", label= "Product Category")
pc_vs_adl_Q4 = transform(pc_vs_adl, by_quarter= True, quarter= "Q4", label= "Product Category")
pc_vs_adl_wholeyear = transform(pc_vs_adl, label= "Product Category")
pc_vs_adl_wholeyear.head(5)
pc_vs_adl_wholeyear.to_csv("pc_vs_adl.csv")
pc_vs_adl_Q1.to_csv("pc_vs_adl_Q1.csv")

C:\Users\quang\AppData\Local\Temp\ipykernel_2116\3153126237.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_vs_adl[["Latest Actual Delivery Date Fiscal Quarter Year", "Quarter"]] = \
C:\Users\quang\AppData\Local\Temp\ipykernel_2116\3153126237.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pc_vs_adl[["Latest Actual Delivery Date Fiscal Quarter Year", "Quarter"]] = \
C:\Users\quang\AppData\Local\Temp\ipykernel_2116\3153126237.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of 

# Numerical Variables

In [8]:
usaid = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0, low_memory= False)
usaid = usaid[["Average Days Late", 
               "Illustrative Price", "Shipped Quantity",
               "Base Unit Multiplier", "Order Cycle Time", "Latest Actual Delivery Date Fiscal Quarter Year", "Latest Actual Delivery Date"]]
# usaid.head(60)
# usaid.isna().sum()
usaid.dropna(inplace= True)
usaid.isna().sum()


Average Days Late                                  0
Illustrative Price                                 0
Shipped Quantity                                   0
Base Unit Multiplier                               0
Order Cycle Time                                   0
Latest Actual Delivery Date Fiscal Quarter Year    0
Latest Actual Delivery Date                        0
dtype: int64